In [1]:
# Load environment variables
from dotenv import load_dotenv
import os

load_dotenv()  # Load the environment variables from a .env file
PRICE_DATA = os.getenv("PRICE_DATA")  # Load the PRICE_DATA variable

In [3]:
import os
from dotenv import load_dotenv
from glob import glob

# Загружаем переменные окружения из файла .env
load_dotenv()

# Получаем значение переменной окружения PRICE_DATA
PRICE_DATA = os.getenv("PRICE_DATA")

# Проверяем, было ли загружено значение
if PRICE_DATA is None:
    print("Ошибка: переменная окружения PRICE_DATA не загружена.")
else:
    print("PRICE_DATA загружена:", PRICE_DATA)

# Теперь ищем все parquet файлы в директории PRICE_DATA
parquet_files = glob(os.path.join(PRICE_DATA, "*.parquet"))

# Проверка найденных файлов
print("Найденные файлы:", parquet_files)

PRICE_DATA загружена: C:/Data course setup/Microsoft VS Code/Project/price_data
Найденные файлы: ['C:/Data course setup/Microsoft VS Code/Project/price_data\\price_data.parquet']


In [6]:
import dask.dataframe as dd

# Загрузка данных из найденного parquet файла
dd_px = dd.read_parquet(parquet_files[0])

# Проверка названий столбцов
print(dd_px.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'ticker'],
      dtype='object')


In [8]:
# Добавление лагов для переменных Close
dd_px['Close_lag'] = dd_px['Close'].shift(1)

# Расчет доходности на основе Close
dd_px['returns'] = (dd_px['Close'] / dd_px['Close_lag']) - 1

# Добавление диапазона High-Low
dd_px['hi_lo_range'] = dd_px['High'] - dd_px['Low']

# Присвоение результата в новый Dask DataFrame
dd_feat = dd_px[['Date', 'Close', 'Close_lag', 'returns', 'hi_lo_range', 'ticker']]

In [9]:
# Конвертация Dask DataFrame в Pandas DataFrame
df = dd_feat.compute()

# Расчет 10-дневной скользящей средней доходности
df['rolling_mean_returns'] = df['returns'].rolling(window=10).mean()

In [10]:
# Вывод первых 5 строк DataFrame
print(df.head())

                       Date      Close  Close_lag   returns  hi_lo_range  \
0 2013-12-02 00:00:00-05:00  17.193993        NaN       NaN     0.421403   
1 2013-12-03 00:00:00-05:00  17.664682  17.193993  0.027375     0.271371   
2 2013-12-04 00:00:00-05:00  17.623508  17.664682 -0.002331     0.261078   
3 2013-12-05 00:00:00-05:00  17.713970  17.623508  0.005133     0.272306   
4 2013-12-06 00:00:00-05:00  17.468174  17.713970 -0.013876     0.223958   

  ticker  rolling_mean_returns  
0   AAPL                   NaN  
1   AAPL                   NaN  
2   AAPL                   NaN  
3   AAPL                   NaN  
4   AAPL                   NaN  


In [12]:
# Load environment variables
from dotenv import load_dotenv
import os

load_dotenv()  # Load the environment variables from a .env file
PRICE_DATA = os.getenv("PRICE_DATA")  # Load the PRICE_DATA variable

import os
from dotenv import load_dotenv
from glob import glob

# Load the environment variables from the .env file
load_dotenv()

# Retrieve the value of the environment variable PRICE_DATA
PRICE_DATA = os.getenv("PRICE_DATA")

# Check if the variable has been loaded successfully
if PRICE_DATA is None:
    print("Error: The environment variable PRICE_DATA has not been loaded.")
else:
    print("PRICE_DATA loaded:", PRICE_DATA)

# Now find all parquet files in the PRICE_DATA directory
parquet_files = glob(os.path.join(PRICE_DATA, "*.parquet"))

# Check the found files
print("Found files:", parquet_files)

import dask.dataframe as dd

# Load data from the found parquet file
dd_px = dd.read_parquet(parquet_files[0])

# Check the column names
print(dd_px.columns)

# Add lags for the Close variable
dd_px['Close_lag'] = dd_px['Close'].shift(1)

# Calculate returns based on Close
dd_px['returns'] = (dd_px['Close'] / dd_px['Close_lag']) - 1

# Add the High-Low range
dd_px['hi_lo_range'] = dd_px['High'] - dd_px['Low']

# Assign the result to a new Dask DataFrame
dd_feat = dd_px[['Date', 'Close', 'Close_lag', 'returns', 'hi_lo_range', 'ticker']]

# Convert Dask DataFrame to Pandas DataFrame
df = dd_feat.compute()

# Calculate the 10-day rolling mean of returns
df['rolling_mean_returns'] = df['returns'].rolling(window=10).mean()

# Output the first 5 rows of the DataFrame
print(df.head())

PRICE_DATA loaded: C:/Data course setup/Microsoft VS Code/Project/price_data
Found files: ['C:/Data course setup/Microsoft VS Code/Project/price_data\\price_data.parquet']
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'ticker'],
      dtype='object')
                       Date      Close  Close_lag   returns  hi_lo_range  \
0 2013-12-02 00:00:00-05:00  17.193993        NaN       NaN     0.421403   
1 2013-12-03 00:00:00-05:00  17.664682  17.193993  0.027375     0.271371   
2 2013-12-04 00:00:00-05:00  17.623508  17.664682 -0.002331     0.261078   
3 2013-12-05 00:00:00-05:00  17.713970  17.623508  0.005133     0.272306   
4 2013-12-06 00:00:00-05:00  17.468174  17.713970 -0.013876     0.223958   

  ticker  rolling_mean_returns  
0   AAPL                   NaN  
1   AAPL                   NaN  
2   AAPL                   NaN  
3   AAPL                   NaN  
4   AAPL                   NaN  


# It was necessary to convert to pandas to calculate the moving average return because Dask does not directly support the rolling window operations needed for this calculation. 
# While Dask is efficient for handling large datasets, certain functions, like rolling averages, are more straightforward to implement with pandas due to its extensive built-in functionality for such operations. 
# In this case, working with pandas allows for simpler and more intuitive syntax to achieve the desired result, even though it means losing some of the parallel processing benefits that Dask provides. 
# Ideally, it would be better to compute the rolling mean directly in Dask to take advantage of its distributed computing capabilities, but this requires more complex handling of rolling windows.